In [1]:
%cd /app

/app


In [2]:
import argparse
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import torch
torch.multiprocessing.set_start_method('spawn')

import jax
from lob.encoding import Vocab, Message_Tokenizer

from lob import inference_no_errcorr as inference
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint

from lob import inference_no_errcorr as inference

2025-02-20 16:57:55.797039: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.6 which is older than the ptxas CUDA version (12.8.61). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2025-02-20 16:57:58.937497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
save_folder = 'data_saved/'
batch_size =2
model_size = 'large'
data_dir ='data/test_set/'
rng_seed = 42
sample_all = False # action='store_true'

In [4]:
stock = 'GOOG'  # 'GOOG', 'INTC'
n_gen_msgs = 500 # how many messages to generate into the future
n_samples = 4

In [5]:
n_messages = 500  # length of input sequence to model
# batch_size = batch_size

v = Vocab()
n_classes = len(v)
seq_len = n_messages * Message_Tokenizer.MSG_LEN
book_dim = 501 #b_enc.shape[1]
book_seq_len = n_messages

n_eval_messages = n_gen_msgs  # how many to load from dataset
eval_seq_len = n_eval_messages * Message_Tokenizer.MSG_LEN

rng = jax.random.key(rng_seed)
rng, rng_ = jax.random.split(rng)

In [6]:
if stock == 'GOOG':
    # ckpt_path = './checkpoints/treasured-leaf-149_84yhvzjt/' # 0.5 y GOOG, (full model)
    ckpt_path = './checkpoints/denim-elevator-754_czg1ss71/' # large model
    # ckpt_path = './checkpoints/stilted-deluge-759_8g3vqor4'  # small model
elif stock == 'INTC':
    # ckpt_path = './checkpoints/pleasant-cherry-152_i6h5n74c/' # 0.5 y INTC, (full model)
    ckpt_path = './checkpoints/eager-sea-755_2rw1ofs3/'  # large model
else:
    raise ValueError(f'stock {stock} not recognized')

In [7]:
print('Loading metadata:', ckpt_path)
args_ckpt = load_metadata(ckpt_path)

Loading metadata: ./checkpoints/denim-elevator-754_czg1ss71/


In [8]:
# scale down to single GPU, single sample inference
args_ckpt.bsz = 1 #1, 10
args_ckpt.num_devices = 1

batchnorm = args_ckpt.batchnorm

In [9]:
# load train state from disk

print('Initializing model...')
new_train_state, model_cls = init_train_state(
    args_ckpt,
    n_classes=n_classes,
    seq_len=seq_len,
    book_dim=book_dim,
    book_seq_len=book_seq_len,
)

print('\nLoading model checkpoint...')
ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']

model = model_cls(training=False, step_rescale=1.0)

param_count = sum(x.size for x in jax.tree_leaves(state.params))
print('param count:', param_count)

Initializing model...
configuring standard optimization setup
[*] Trainable Parameters: 35776312

Loading model checkpoint...
param count: 35776312


/tmp/ipykernel_689544/2594640255.py:22: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  param_count = sum(x.size for x in jax.tree_leaves(state.params))


In [10]:
data_dir = data_dir + stock
# data_dir = data_dir + 'GOOG'
print(f"Directory Path: {data_dir}")

Directory Path: data/test_set/GOOG


In [11]:
# Using home directory
from pathlib import Path
import os

# data_dir = Path.home() / "data/test_set/GOOG"
# Or use a relative path from current working directory
# data_dir = Path("data/test_set/GOOG")
data_dir = Path(data_dir)

Path(data_dir).mkdir(parents=True, exist_ok=True)
folder_path = Path(data_dir)
file_count = len([f for f in folder_path.iterdir() if f.is_file()])
print(f"There are {file_count} files in the folder {str(data_dir)}.")

There are 36 files in the folder data/test_set/GOOG.


In [12]:
print(n_messages, n_eval_messages)

500 500


In [13]:
from pathlib import Path
import os

# Print current working directory to help verify the path
print(f"Current working directory: {os.getcwd()}")

# Use relative path to data/test_set/GOOG
data_dir = Path("data/test_set/GOOG")

try:
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    print(f"Successfully created or verified directory: {data_dir}")
    
    file_count = len([f for f in Path(data_dir).iterdir() if f.is_file()])
    print(f"There are {file_count} files in the folder {data_dir}.")
except Exception as e:
    print(f"Error: {str(e)}")

Current working directory: /app
Successfully created or verified directory: data/test_set/GOOG
There are 36 files in the folder data/test_set/GOOG.


In [14]:
ds = inference.get_dataset(data_dir, n_messages, n_eval_messages)

In [15]:
save_folder

'data_saved/'

In [16]:
print('Generating...')
inference.sample_new(
    n_samples,
    batch_size,
    ds,
    rng,
    seq_len,
    n_messages,
    n_gen_msgs,
    state,
    model,
    batchnorm,
    v.ENCODING,
    stock,
    save_folder=save_folder + '/' + stock + '/',
    sample_all=sample_all,
)
print('DONE.')

Generating...


  0%|          | 0/2 [00:00<?, ?it/s]

BATCH [4497, 6855]
!!!
book_l2_init - shape (2, 40)


m_seq_raw_inp - shape (2, 500, 14)


BEFORE generate_batched(...)


sample_top_n - -1


tick_size - 100


m_seq_inp - shape (2, 11000): [[ 1005 12010 12007 ...   103   518   863]
 [ 1003 12009 12008 ...   785    50   388]]


b_seq_inp - shape (2, 500, 501): [[[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [-1.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]]


n_gen_msgs - 500


sim_states_init - LobState(asks=Array([[[   884800,        25, 815683086,         0,     53062,
         256032794],
        [   885000,       100, 815712022,         0,     53063,
         240275783],
        [   884700,        25,   -900004,   -900000,         0,
 

  0%|          | 0/2 [06:34<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 print('Generating...')                                                                      │
│ ❱  2 inference.sample_new(                                                                       │
│    3 │   n_samples,                                                                              │
│    4 │   batch_size,                                                                             │
│    5 │   ds,                                                                                     │
│                                                                                                  │
│ /app/lob/inference_no_errcorr.py:1149 in sample_new                                              │
│                                                                                                  │
│   1146 │   │   print(f'sim_states_init - {sim_states_init}\n\n')                                 │
│   1147 │   │   print(f'jax.random.split(rng_, batch_size) - {jax.random.split(rng_, batch_size)  │
│   1148 │   │                                                                                     │
│ ❱ 1149 │   │   m_seq_gen, b_seq_gen, msgs_decoded, l2_book_states, num_errors = generate_batche  │
│   1150 │   │   │   sim_init,                                                                     │
│   1151 │   │   │   train_state,                                                                  │
│   1152 │   │   │   model,                                                                        │
│                                                                                                  │
│ /opt/conda/envs/myenv/lib/python3.12/site-packages/jax/_src/traceback_util.py:179 in             │
│ reraise_with_filtered_traceback                                                                  │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                                                                  │
│ /opt/conda/envs/myenv/lib/python3.12/site-packages/jax/_src/pjit.py:298 in cache_miss            │
│                                                                                                  │
│    295                                                                                           │
│    296   @api_boundary                                                                           │
│    297   def cache_miss(*args, **kwargs):                                                        │
│ ❱  298 │   outs, out_flat, out_tree, args_flat, jaxpr, attrs_tracked = _python_pjit_helper(      │
│    299 │   │   jit_info, *args, **kwargs)                                                        │
│    300 │   executable = _read_most_recent_pjit_call_executable(jaxpr)                            │
│    301 │   maybe_fastpath_data = _get_fastpath_data(                                             │
│                                                            

# ------------------------------------
# Lob_bench - analyze generated data

In [17]:
import sys
sys.path.append('/app/lob_bench')

In [18]:
import glob
import data_loading
import impact
import matplotlib.pyplot as plt

In [ ]:
message_paths = sorted(glob.glob('/app/data_saved/GOOG/data_gen/GOOG*message*.csv'))
book_paths = sorted(glob.glob('/app/data_saved/GOOG/data_gen/GOOG*orderbook*.csv'))

print(len(book_paths))

In [20]:
m_seqs = [data_loading.load_message_df(path) for path in message_paths]
b_seqs = [data_loading.load_book_df(path) for path in book_paths]

In [21]:
mb_seqs=[impact._filter_touch_events(m,b) for m,b in zip(m_seqs,b_seqs)]

In [ ]:
res=impact.macro_impact_analyse(3*60,100,32,m_seqs,b_seqs)

In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.plot(res[1][:],res[0][:], 'o', c='blue', alpha=0.5, markeredgecolor='none')
ax.set_title("GOOG Real Long Form")
ax.set_ylabel("M_i")
ax.set_xlabel("P_i")
#ax.set_ylim([-2,2])
#ax.set_yscale('log')
#ax.set_xscale('log')

In [ ]:
res[0][:]

In [28]:
# root_path="/data1/sascha/data/sqrt/GOOG/"

root_path="/app/data_saved/GOOG/"

loader = data_loading.Simple_Loader(
            real_data_path= root_path+"data_real",
            gen_data_path= root_path+"data_gen",
            cond_data_path=root_path+"data_cond",
)

In [ ]:
df=impact.macro_impact_compare(100000,100,32,loader)
df
